## **Predicting Disease Spread: Model Prophet**

### **Imports**

In [3]:
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

sys.path.append(os.path.abspath(".."))
from src.data_setup import load_data, split_cities
from src.prophet import (
    forecast_dengue_cases
)

from prophet import Prophet

In [4]:
features_train, labels_train = load_data(train=True)
sj_features_train, iq_features_train = split_cities(features_train)
sj_labels_train, iq_labels_train = split_cities(labels_train)
features_test, labels_test = load_data(train=False)
sj_features_test, iq_features_test = split_cities(features_test)
iq_features_train_num = iq_features_train.drop(columns=['week_start_date'])
iq_features_test_num = iq_features_test.drop(columns=['week_start_date'])
sj_features_train_num = sj_features_train.drop(columns=['week_start_date'])
sj_features_test_num = sj_features_test.drop(columns=['week_start_date'])

### **Predictions (no tunning)**

In [ ]:
X_train_dict = {'sj': sj_features_train, 'iq': iq_features_train}
y_train_dict = {'sj': sj_labels_train, 'iq': iq_labels_train}
X_test_dict = {'sj': sj_features_test, 'iq': iq_features_test}

city_predictions = forecast_dengue_cases(X_train_dict, y_train_dict, X_test_dict)
all_predictions = []
for city, preds in city_predictions.items():
    city_test = X_test_dict[city]
    
    city_results = pd.DataFrame({
        'city': city,
        'year': city_test.index.get_level_values('year'),
        'weekofyear': city_test.index.get_level_values('weekofyear'),
        'total_cases': preds
    })
    
    all_predictions.append(city_results)

final_predictions = pd.concat(all_predictions)

final_predictions['total_cases'] = np.round(final_predictions['total_cases']).clip(0).astype(int)

print("Final predictions shape:", final_predictions.shape)
print(final_predictions.head())



Outputs:


SJ  
Best parameters found: {'changepoint_prior_scale': 0.01, 'daily_seasonality': False, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 0.01, 'weekly_seasonality': True, 'yearly_seasonality': True}
Best validation MAE: 27.16


IQ  
Best parameters found: {'changepoint_prior_scale': 0.05, 'daily_seasonality': False, 'holidays_prior_scale': 1.0, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 0.01, 'weekly_seasonality': True, 'yearly_seasonality': True}
Best validation MAE: 6.41


In [18]:
final_predictions.to_csv('../data/predictions/prophet_htun.csv', index=False)